# Assignment 3.1. Sequence Classification

# Task: Aspect-level Sentiment Classification(10pt)

Reading material:
- [1] R. He, WS. Lee & D. Dahlmeier. Exploiting document knowledge for aspect-level sentiment classification. 2018. https://arxiv.org/abs/1806.04346.


Build an attention-based aspect-level sentiment classification model with biLSTM. Your model shall include:

- BiLSTM network that learns sentence representation from input sequences.
- Attention network that assigns attention score over a sequence of biLSTM hidden states based on aspect terms representation.
- Fully connected network that predicts sentiment label, given the representation weighted by the attention score.

Requirements:

- You shall train your model bsaed on transferring learning. That is, you need first train your model on documnet-level examples. Then the learned weights will be used to initialize aspect-level model and fine tune it on aspect-level examples.
- You shall use the alignment score function in attention network as following expression:$$f_{score}(h,t)=tanh(h^TW_a t)$$
- You shall evaluate the trained model on the provided test set and show the accuracy on test set.


In [53]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [0]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [0]:
import _pickle as cPickle

# Load Data

In [0]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [0]:
aspect_path = '/drive/My Drive/Colab Notebooks/DeepLearning/Practical5/practice-5-data/aspect-level/'


vocab = read_pickle(aspect_path, 'all_vocab.pkl')

train_x = read_pickle(aspect_path, 'train_x.pkl')
train_y = read_pickle(aspect_path, 'train_y.pkl')
dev_x = read_pickle(aspect_path, 'dev_x.pkl')
dev_y = read_pickle(aspect_path, 'dev_y.pkl')
test_x = read_pickle(aspect_path, 'test_x.pkl')
test_y = read_pickle(aspect_path, 'test_y.pkl')

train_aspect = read_pickle(aspect_path, 'train_aspect.pkl')
dev_aspect = read_pickle(aspect_path, 'dev_aspect.pkl')
test_aspect = read_pickle(aspect_path, 'test_aspect.pkl')


pretrain_data = read_pickle(aspect_path, 'pretrain_data.pkl')
pretrain_label = read_pickle(aspect_path, 'pretrain_label.pkl')

In [0]:
class Dataiterator_doc():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y
        
class Dataiterator_aspect():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, aspect_data, seq_length=32, decoder_dim=300, batch_size=32):
        
        len_aspect_data = len(aspect_data[0])
        #self.len_doc_data = len(doc_data[0])
        
        self.X_aspect = aspect_data[0] 
        self.y_aspect = aspect_data[1]
        self.aspect_terms = aspect_data[2]  
        self.num_data = len_aspect_data
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X_aspect = self.X_aspect[np.array(X_ids)] # X values (encoder input) per batch
        batch_y_aspect = self.y_aspect[np.array(X_ids)] # y_in values (decoder input) per batch
        batch_aspect_terms = self.aspect_terms[np.array(X_ids)]
        
        return batch_X_aspect, batch_y_aspect, batch_aspect_terms

          
    def all(self): # return all data examples
        return self.X_aspect, self.y_aspect, self.aspect_terms

In [0]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Lambda, Dropout, LSTM,Bidirectional
from keras.layers import Reshape, Activation, RepeatVector, concatenate, Concatenate, Dot, Multiply
import keras.backend as K
from keras.engine.topology import Layer
from keras import initializers
from keras import regularizers
from keras import constraints


In [0]:
overal_maxlen = 82
overal_maxlen_aspect = 7

# Define Attention Network Layer
- Define class for Attention Layer
- You need to finish the code for calculating the attention weights

In [0]:

class Attention(Layer):
    def __init__(self,  **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
       
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        super(Attention, self).__init__(**kwargs)


    def build(self, input_shape):
        # print('build input_shape=',input_shape)
        assert type(input_shape) == list
       
        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),)

        self.built = True
        # print('finish build input_shape=',input_shape)


    def compute_mask(self, input_tensor, mask=None):
        # print('compute mask input_tensor=',input_tensor)
        
        assert type(input_tensor) == list
        assert type(mask) == list
        
        # print('finish compute mask input_tensor=', input_tensor)
        return None


    def call(self, input_tensor, mask=None):
        # print('call input_tensor=',input_tensor)
        
        x = input_tensor[0]
        # print('x ', x._keras_shape)
        aspect = input_tensor[1]
        # print(aspect._keras_shape)
        mask = mask[0]
        # print(mask.shape)
        ###YOUR CODE HERE###
        aspect = K.transpose(K.dot(self.W, K.transpose(aspect))) #W_a
        # print('W_a ', aspect.shape)
        aspect = K.expand_dims(aspect, axis=-2)
        # print('W_a expand ', aspect.shape)
        aspect = K.repeat_elements(aspect, self.steps, axis=1)
        # print('W_a repeat ', aspect.shape)
        eij = K.sum(x*aspect, axis=-1)  #h*W_a*t
        # print('hWat ',eij.shape)

        eij = K.tanh(eij)   #beta_i
        # print('beta_i ',eij.shape)
        a = K.exp(eij)    #exp(beta_i)
        # print('exp(beta_i) ',a.shape)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())    #a_i
        # print('call a shape=',a.shape)
        # print('a=', a)

        return a

   
    def compute_output_shape(self, input_shape):

        return (input_shape[0][0], input_shape[0][1])
    





In [0]:
class Average(Layer):
  
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x,mask=None):
        if self.mask_zero:           
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

# Establish computation Grah for model
- Input tensors
- Shared WordEmbedding layer 
- Attention network layer  
- Shared BiLSTM layer
- Shared fully connected layer(prediction layer)


In [0]:
dropout = 0.5     
recurrent_dropout = 0.1
vocab_size = len(vocab)
num_outputs = 3 # labels

## Input tensors

In [0]:
#YOUR CODE HERE ##### Inputs #####
sentence_input = Input(shape=(overal_maxlen,), dtype='int32', name='sentence_input')  #input for sentence from aspect-level data
aspect_input = Input(shape=(overal_maxlen_aspect,), dtype='int32', name='aspect_input') #input for aspect terms
pretrain_input = Input(shape=(None,), dtype='int32', name='pretrain_input')             #input for sentence from document-level data

## Shared WordEmbedding layer

In [63]:
#YOUR CODE HERE### represent aspect as averaged word embedding ###
word_emb = Embedding(vocab_size, 300, mask_zero=True, name='word_emb')

print ('use average term embs as aspect embedding')
aspect_term_embs = word_emb(aspect_input)
aspect_embs = Average(mask_zero=True, name='aspect_emb')(aspect_term_embs)

use average term embs as aspect embedding


In [0]:
#YOUR CODE HERE ### sentence representation from embedding ###
sentence_embs = word_emb(sentence_input)
pretrain_embs = word_emb(pretrain_input)

## Shared BiLSTM layer

In [65]:
#YOUR CODE HERE ### sentence representation from embedding ###
print ('use a BiLSTM layer')
rnn = Bidirectional(LSTM(300, return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout, name='lstm'))
sentence_output = rnn(sentence_embs)
pretrain_output = rnn(pretrain_embs)
print(sentence_output.shape)

use a BiLSTM layer
(None, 82, 600)


## Attention Layer

In [0]:
class WeightedSum(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(WeightedSum, self).__init__(**kwargs)
        # print('_init_')

    def call(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list

        x = input_tensor[0]
        a = input_tensor[1]

        a = K.expand_dims(a)
        weighted_input = x * a
        # print('call')
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        # print('compute_output_shape')
        return (input_shape[0][0], input_shape[0][-1])

    def compute_mask(self, x, mask=None):
        # print ('compute_mask')
        return None

In [0]:
##YOUR CODE HERE
att_weights = Attention(name='att_weights')([sentence_output, aspect_embs])
att_contex = WeightedSum()([sentence_output, att_weights])

## Prediction Layer

In [0]:
#YOUR CODE HERE
pretrain_avg = Average(mask_zero=True)(pretrain_output)

sentence_output = Dense(num_outputs, name='dense_1')(att_contex)
pretrain_output = Dense(num_outputs, name='dense_2')(pretrain_avg)

In [0]:
aspect_probs = Activation('softmax', name='aspect_model')(sentence_output)
doc_probs = Activation('softmax', name='pretrain_model')(pretrain_output)

# Build Models for document-level and aspect-level data
- The two models shared the embedding, BiLSTM, Prediction Layer

In [0]:
### YOUR CODE HERE
model1 = Model(inputs=[pretrain_input], outputs=[doc_probs])

In [0]:
model2 = Model(inputs=[sentence_input, aspect_input], outputs=[aspect_probs])

# Train Model
- First Train model on document-level data.
- Then Train  model on aspect-level data

## Train on document-level data

In [0]:

import keras.optimizers as opt
optimizer=opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 128
train_steps_epoch = len(pretrain_data)/batch_size
batch_train_iter_doc = Dataiterator_doc(pretrain_data, pretrain_label, batch_size)

In [0]:
###YOUR CODE HERE###
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter):
    
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[pretrain_X, pretrain_y] for pretrain_X, pretrain_y in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch
                

                
    history = model.fit_generator(train_gen(), \
                                  steps_per_epoch=train_steps_epoch, \
                                  epochs = 10, callbacks = earlystop_callbacks)

In [94]:
train_generator(model1, batch_train_iter_doc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
235/234 [==============================] - 469s 2s/step - loss: 0.9292 - categorical_accuracy: 0.5689
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,categorical_accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


235/234 [==============================] - 469s 2s/step - loss: 0.7738 - categorical_accuracy: 0.6471
Epoch 3/10
235/234 [==============================] - 466s 2s/step - loss: 0.7481 - categorical_accuracy: 0.6703
Epoch 4/10
235/234 [==============================] - 466s 2s/step - loss: 0.7208 - categorical_accuracy: 0.6811
Epoch 5/10
235/234 [==============================] - 463s 2s/step - loss: 0.6443 - categorical_accuracy: 0.7203
Epoch 6/10
235/234 [==============================] - 467s 2s/step - loss: 0.6606 - categorical_accuracy: 0.7144
Epoch 7/10
235/234 [==============================] - 464s 2s/step - loss: 0.6712 - categorical_accuracy: 0.7098
Epoch 8/10
235/234 [==============================] - 467s 2s/step - loss: 0.6617 - categorical_accuracy: 0.7168
Epoch 9/10
235/234 [==============================] - 463s 2s/step - loss: 0.5907 - categorical_accuracy: 0.7512
Epoch 10/10
235/234 [==============================] - 463s 2s/step - loss: 0.6197 - categorical_accuracy: 

## Train on aspect-level data

In [0]:
train_steps_epoch = len(train_x)/batch_size
batch_train_iter_aspect = Dataiterator_aspect([train_x, train_y, train_aspect], batch_size)
val_steps_epoch = len(dev_x)/batch_size
batch_val_iter_aspect = Dataiterator_aspect([dev_x, dev_y, dev_aspect], batch_size)

import keras.optimizers as opt
optimizer = opt.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=10, clipvalue=0)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [0]:
### YOUR CODE HERE
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter, batch_val_iter):
    
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[[X, aspect], [y]] for X, y, \
                             aspect in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch
                
    def val_gen():
        while True:
            val_batches = [[[X, aspect], [y]] for X, y, \
                           aspect in batch_val_iter]
            for val_batch in val_batches:
                yield val_batch
                
    history2 = model.fit_generator(train_gen(), validation_data=val_gen(), \
                                  validation_steps=val_steps_epoch, steps_per_epoch=train_steps_epoch, \
                                  epochs = 10, callbacks = earlystop_callbacks)



In [100]:
train_generator(model2, batch_train_iter_aspect, batch_val_iter_aspect)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15/14 [===============================] - 10s 663ms/step - loss: 1.0725 - categorical_accuracy: 0.4563 - val_loss: 1.0352 - val_categorical_accuracy: 0.6562
Epoch 2/10
15/14 [===============================] - 9s 588ms/step - loss: 0.9826 - categorical_accuracy: 0.6375 - val_loss: 1.0043 - val_categorical_accuracy: 0.6719
Epoch 3/10
15/14 [===============================] - 9s 590ms/step - loss: 0.9247 - categorical_accuracy: 0.6062 - val_loss: 0.8724 - val_categorical_accuracy: 0.6328
Epoch 4/10
15/14 [===============================] - 9s 580ms/step - loss: 0.8721 - categorical_accuracy: 0.6479 - val_loss: 0.8567 - val_categorical_accuracy: 0.6172
Epoch 5/10
15/14 [===============================] - 9s 585ms/step - loss: 0.8431 - categorical_accuracy: 0.6458 - val_loss: 0.7317 - val_categorical_accuracy: 0.7031
Epoch 6/10
15/14 [===============================] - 9s 574ms/step - loss: 0.8447 - categorical_accuracy: 0.6333 - val_loss: 0.6639 - val_categorical_accuracy: 0.66

## Evaluating on test set
- show the accuracy

In [0]:
# ##YOUR CODE HERE
# from keras.callbacks import EarlyStopping, ModelCheckpoint

# test_steps_epoch = len(test_x)/batch_size
# batch_test_iter = Dataiterator_aspect([test_x, test_y, test_aspect], batch_size)

# def test_gen():
#     while True:
#         train_batches = [[[X, aspect], [y]] for X, y, \
#                           aspect in batch_test_iter]
#         for train_batch in train_batches:
#             yield train_batch
# 
# results = model2.evaluate(test_gen(), steps=test_steps_epoch)

In [115]:
##YOUR CODE HERE
result = model2.evaluate([test_x, test_aspect], [test_y])

638/638 [==============================] - 2s 3ms/step


In [0]:
model2.metrics_names

['loss', 'categorical_accuracy']

In [120]:
print('loss on test set is: ', result[0])
print('categorical accuracy on test set is: ', result[1])

loss on test set is:  0.839440485713624
categorical accuracy on test set is:  0.5909090638160706
